In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np
import requests
import time
import re
import dateutil.parser
from functions import (get_sector, changes_from_press, get_content_header, 
                       get_df, normalise_to_index, save_data)
from selenium import webdriver


C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
year = datetime.today().year

driver = webdriver.Firefox()
driver.get(f'https://servicenow.gcs-web.com/news-releases?field_nir_news_date_value%5Bmin%5D=2023#views-exposed-form-widget-news-widget-news-ul')

data = []

ticker = 'now'
sector = get_sector(ticker)
index_ticker = '^GSPC'
 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

year = datetime.today().year

while True:
    if year == 2013:
        break
    
    html = requests.get(f'https://servicenow.gcs-web.com/news-releases?field_nir_news_date_value%5Bmin%5D={year}#views-exposed-form-widget-news-widget-news-ul')


    time.sleep(2)
    soup = BeautifulSoup(html.text) 
    table = soup.find('tbody')
    
    articles = table.find_all('tr')
    
    for article in articles:
        
        title_block = article.find('td', attrs={'class':'cus_title'})
        
        date = article.find('time', attrs={'class':'datetime'}).text
        title = title_block.find('a').text.rstrip() 
        url = title_block.find('a').get('href')
        url = f'https://servicenow.gcs-web.com{url}'
        
        title = re.sub("\s\s+", " ", title)

        date = dateutil.parser.parse(date).date()
        
        content = article.find('div', {'class':'nir-news-table-teaser summary-hidden'}).text.rstrip().lstrip()
        
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    year = year - 1
    

driver.close()

save_data(data, ticker, sector)

In [17]:
article

<tr>
<td headers="view-field-nir-news-date-table-column"><a class="summary-toggle nir-news-widget-summary-toggle-date" href="#"><span>Toggle Summary</span></a><time class="datetime" datetime="12Z" timezone="America/New_York">Jan 2, 2014</time>
</td>
<td class="cus_title">
<a href="/news-releases/news-release-details/servicenow-announce-fourth-quarter-and-fiscal-year-2013" hreflang="en">ServiceNow to Announce Fourth Quarter and Fiscal Year 2013 Financial Results on Wednesday, January 29, 2014</a>
<div class="nir-news-table-teaser summary-hidden">
  SANTA CLARA, Calif. --(BUSINESS WIRE)--Jan. 2, 2014-- ServiceNow (NYSE: NOW), the enterprise IT cloud company, today announced it will release financial results for the fourth quarter and fiscal year ended December 31, 2013 following the close of market on Wednesday, January 29, 2014 .
</div>
</td><td headers="view-field-nir-news-title-table-column">
<a href="/news-releases/news-release-details/servicenow-announce-fourth-quarter-and-fiscal-ye